<a href="https://colab.research.google.com/github/ellenwterry/Applications/blob/master/PricingAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
from gspread_dataframe import set_with_dataframe

import numpy as np
import pandas as pd
import patsy
from sklearn.linear_model import LogisticRegression



!pip install pystan
!pip install corner
import stan

!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
project = 'bayesian-playground' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

job = client.get_job('script_job_b807e2720876a68a03c7e32cf2dd14eb_6') # Job ID inserted based on the query results selected to explore
quoteData = job.to_dataframe()

quoteData.to_csv('quoteData.csv')
from google.colab import files
files.download("quoteData.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
quoteData = quoteData.reset_index()
quoteData = quoteData.rename(columns={"index": "SampleID"})
quoteData['Result']=quoteData.Result.astype('int32')
quoteData['RSF']=quoteData.RSF.astype('int32')
quoteData['QuoteDiff']=quoteData.QuoteDiff.astype('int32')/1000
quoteData['RFPDiff']=quoteData.RFPDiff.astype('int32')
quoteData['ATPDiff']=quoteData.ATPDiff.astype('int32')

quoteTrain = quoteData.sample(n=len(quoteData.index)-100)
# hold out 100 for testing
values_list = quoteTrain['SampleID']
quoteTest = quoteData[~quoteData['SampleID'].isin(values_list)]

In [ ]:
rows = quoteTrain.count()[0]
columns = quoteTrain[quoteTrain.columns[0]].count()

In [ ]:
m  = rows
# Create a formula object
#formula = 'Result ~ RSF + QuoteDiff + RFPDiff + ATPDiff'



In [ ]:
# Create the model matrix
Xmatrix = patsy.dmatrix('RSF + QuoteDiff + RFPDiff + ATPDiff', quoteTrain)
yDf = (quoteTrain['Result']).astype('int32')
yArray = quoteTrain['Result'].values.astype('int32')

rows = Xmatrix.shape[0]
columns = Xmatrix.shape[1]
columns, rows

(5, 6228)

In [ ]:
# Create a LogisticRegression object
model = LogisticRegression()
model.fit(Xmatrix, yDf)
Pred = model.predict(Xmatrix)
Probs = pd.DataFrame(model.predict_proba(Xmatrix))
theta = np.matrix(model.coef_)
prob2 = np.exp((np.dot(theta,Xmatrix.transpose())).transpose())/(1+np.exp((np.dot(theta,Xmatrix.transpose())).transpose()))

$P({Y})=exp({\beta_0 + \beta_n}X_n) / exp(1+ exp({\beta_0 + \beta}X_n)$

In [ ]:
priors = theta
yTrain = yDf

In [ ]:
aTheta = np.array(theta)


In [ ]:
stanMod = """
data {
  int N_train;
  int K;
  int y_train[N_train];
  matrix[N_train, K] x_train;
  real p_b[K];
  real<lower = 0> p_sb[K];
}
parameters {
  vector[K] beta;
}
transformed parameters {
  vector[N_train] y_hat;
  for(n in 1:N_train)
    y_hat[n] = x_train[n]*beta;
}
model {
  target += normal_lpdf(beta | p_b, p_sb);
  target += bernoulli_lpmf(y_train | inv_logit(y_hat));
}
"""


In [ ]:
mData = {"N_train": rows,
         "K": 5,
         "y_train": yArray,
         "x_train": Xmatrix,
         "p_b" : [-0.00081737,  0.65316983,  0.18141612,  0.04508703,  0.01720372],
         "p_sb" : np.repeat(.2,5),
         }

In [ ]:
postr = stan.build(stanMod, data = mData, random_seed = 1)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_k8hjh86s/model_gxvl5hdt.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_k8hjh86s/model_gxvl5hdt.stan', line 7, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_k8hjh86s/model_gxvl5hdt.stan', line 8, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format

In [ ]:
fit = postr.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   1% (101/8000)
Sampling:   2% (200/8000)
Sampling:   4% (300/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1901/8000)
Sampling:  25% (2002/8000)
Sampling:  26% (2101/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Sampling:  42% (3400/8000)
Sampling:  44% (3500/8000)
S

In [ ]:
df = fit.to_frame()

In [ ]:
Params = df.describe().T
beta1 = Params['mean']['beta.1']
beta2 = Params['mean']['beta.2']
beta3 = Params['mean']['beta.3']
beta4 = Params['mean']['beta.4']
beta5 = Params['mean']['beta.5']


In [ ]:
thetaStan = np.array([beta1, beta2, beta3, beta4, beta5])

In [ ]:
XmatrixTest = patsy.dmatrix('RSF + QuoteDiff + RFPDiff + ATPDiff', quoteTest)
yDfTest = (quoteTest['Result']).astype('int32')
yArray = quoteTest['Result'].values.astype('int32')

In [ ]:
probTest = np.exp((np.dot(thetaStan,XmatrixTest.transpose())).transpose())/(1+np.exp((np.dot(thetaStan,XmatrixTest.transpose())).transpose()))
probTest

array([0.35534923, 0.86756098, 0.84769444, 0.68311812, 0.28937576,
       0.30297757, 0.26648771, 0.68311812, 0.96559699, 0.03529948,
       0.32073071, 0.84769444, 0.41873157, 0.84769444, 0.11557807,
       0.18040506, 0.61149027, 0.30017345, 0.49674858, 0.28765614,
       0.43748724, 0.59159161, 0.60527822, 0.86532144, 0.67015384,
       0.40161843, 0.71513902, 0.23735559, 0.41834979, 0.9794205 ,
       0.61640272, 0.44425591, 0.99384295, 0.61149027, 0.57371744,
       0.63121563, 0.65817872, 0.6247258 , 0.76303352, 0.81628528,
       0.08530635, 0.5511286 , 0.0619119 , 0.51630156, 0.9423548 ,
       0.74918249, 0.63121563, 0.22984789, 0.37578025, 0.97361629,
       0.96759631, 0.90522285, 0.74733147, 0.9969014 , 0.77469659,
       0.91952259, 0.54305444, 0.21824427, 0.38418854, 0.71372442,
       0.7077305 , 0.90523374, 0.76002862, 0.6120877 , 0.64197218,
       0.67397822, 0.57118749, 0.89575633, 0.78104024, 0.63121563,
       0.6247258 , 0.65721008, 0.60772418, 0.16829857, 0.18040

In [ ]:
quoteTest

,SampleID,Customer_ID,RSF,RSFDesc,QuoteDiff,RFPDiff,ATPDiff,Result,Product_ID,Quote,PredProb
27,27,C7,2,SILVER,-2.5,0,7,0,LS3410,94500,0.355349
40,40,C7,2,SILVER,8.5,0,38,1,HS2323,108500,0.867561
42,42,C10,2,SILVER,8.5,0,28,1,HS2323,108500,0.847694
99,99,C6,1,GREEN,13.9,0,-58,0,LS2100,88500,0.683118
149,149,C8,1,GREEN,-2.5,4,9,0,LS3410,94500,0.289376
...,...,...,...,...,...,...,...,...,...,...,...
5861,5861,C6,1,GREEN,3.5,0,27,0,HS2323,108500,0.572103
5908,5908,C5,4,PLATNIUM,2.0,0,3,1,HS2200,106000,0.763864
6005,6005,C4,4,PLATNIUM,-2.3,4,-17,0,LS7620,112300,0.563900
6014,6014,C5,4,PLATNIUM,-2.3,0,-96,0,LS7620,112300,0.229848


In [ ]:
condition = quoteTest['PredProb'] > .5

# Insert a new column
quoteTest['PredResult'] = np.where(condition, int(1),  int(0))


<ipython-input-33-439718c81859>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quoteTest['PredResult'] = np.where(condition, int(1),  int(0))


In [ ]:
quoteTest

,SampleID,Customer_ID,RSF,RSFDesc,QuoteDiff,RFPDiff,ATPDiff,Result,Product_ID,Quote,PredProb,1,PredResult
27,27,C7,2,SILVER,-2.5,0,7,0,LS3410,94500,0.355349,0.0,0
40,40,C7,2,SILVER,8.5,0,38,1,HS2323,108500,0.867561,0.0,1
42,42,C10,2,SILVER,8.5,0,28,1,HS2323,108500,0.847694,0.0,1
99,99,C6,1,GREEN,13.9,0,-58,0,LS2100,88500,0.683118,0.0,1
149,149,C8,1,GREEN,-2.5,4,9,0,LS3410,94500,0.289376,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,5861,C6,1,GREEN,3.5,0,27,0,HS2323,108500,0.572103,0.0,1
5908,5908,C5,4,PLATNIUM,2.0,0,3,1,HS2200,106000,0.763864,0.0,1
6005,6005,C4,4,PLATNIUM,-2.3,4,-17,0,LS7620,112300,0.563900,0.0,1
6014,6014,C5,4,PLATNIUM,-2.3,0,-96,0,LS7620,112300,0.229848,0.0,0


In [ ]:
import sklearn.metrics as metrics
confusion_matrix = metrics.confusion_matrix(quoteTest['Result'], quoteTest['PredResult'])
print(confusion_matrix)
accuracy = metrics.accuracy_score(quoteTest['Result'], quoteTest['PredResult'])
print(accuracy)

[[31 19]
 [ 3 47]]
0.78


In [ ]:
import sklearn.metrics as metrics
confusion_matrix = metrics.confusion_matrix(quoteTest['Result'], quoteTest['PredResult'])
print(confusion_matrix)
accuracy = metrics.accuracy_score(quoteTest['Result'], quoteTest['PredResult'])
print(accuracy)